# Preprocessing:

In [ ]:
import sys
print(sys.version)
#3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
!pip install contractions==0.1.73
!pip install emoji==2.8.0
!pip install nltk==3.8.1
!pip install pingouin==0.5.4

In [ ]:
#Import statements
import pandas as pd
import numpy as np
import re
import nltk
import emoji
import contractions
import string
import pingouin as pg

pd.set_option('display.max_columns', None)

## ICC


In [ ]:
####import data from google colab
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')


In [ ]:
label_1 = pd.read_excel("/content/drive/MyDrive/codeAndData/data/Label1_Student.xlsx")
label_2 = pd.read_excel("/content/drive/MyDrive/codeAndData/data/Label2_X.xlsx")

In [ ]:
emotions_columns = ['Attentive','Alert','Determined', 'Inspired', 'Active', 'Hostile', 'Ashamed', 'Upset', 'Afraid', 'Nervous']
label_1=label_1[emotions_columns]
label_2=label_2[emotions_columns]
label_1 = label_1.astype(float)
label_2 = label_2.astype(float)

In [ ]:
#check data mistake or missing
## first check : 0-5
np.sum((label_1 < 0) | (label_1 > 5))

In [ ]:
np.sum((label_2 < 0) | (label_2 > 5))

In [ ]:
def check_all_zero(df_label):
  rows_with_any_zero = df_label.index[df_label.eq(0).all(axis=1)].tolist()
  return df_label.loc[rows_with_any_zero].sum(axis=1)

In [ ]:
np.sum(check_all_zero(label_1))

In [ ]:
np.sum(check_all_zero(label_2))

In [ ]:
label_1_zero_indices = label_1.index[label_1.eq(0).all(axis=1)].tolist()
label_2_zero_indices = label_2.index[label_2.eq(0).all(axis=1)].tolist()

In [ ]:
len(label_2_zero_indices)

In [ ]:
len(label_1_zero_indices)

In [ ]:
#0 data useless no any emotion
combined_indices = set(label_1_zero_indices + label_2_zero_indices)
len(combined_indices)

In [ ]:
label_1 = label_1.drop(combined_indices)
label_2 = label_2.drop(combined_indices)

In [ ]:

def calculate_icc(label_1, label_2, variable_name):

    all_indexes = label_1.index
    data_rows = []

    for idx in all_indexes:
        data_rows.append({'Index': idx, 'Labeler': 'Label_1', 'Score': label_1.loc[idx, variable_name]})
    for idx in all_indexes:
        data_rows.append({'Index': idx, 'Labeler': 'Label_2', 'Score': label_2.loc[idx, variable_name]})

    icc_dataframe = pd.DataFrame(data_rows)

    icc_result = pg.intraclass_corr(data=icc_dataframe, targets='Index', raters='Labeler', ratings='Score',nan_policy='omit').round(4)
    return icc_result.set_index("Type")



In [ ]:
ICC_results = pd.DataFrame()
for column in emotions_columns:
    print(column)
    result = calculate_icc(label_1, label_2, column)
    print(result)
    ICC_results = pd.concat([ICC_results, result], ignore_index=True)
    ICC_results.loc[len(ICC_results)] = column



In [ ]:
path = f'/content/drive/MyDrive/codeAndData/data/Test_results/ICC_results.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  ICC_results.to_csv(f,index=False)

## To get dataset

In [ ]:
data_1 = pd.read_excel("/content/drive/MyDrive/codeAndData/data/Label1_Student.xlsx")
data_2 = pd.read_excel("/content/drive/MyDrive/codeAndData/data/Label2_X.xlsx")

In [ ]:
emotions_columns = ['Attentive','Alert','Determined', 'Inspired', 'Active', 'Hostile', 'Ashamed', 'Upset', 'Afraid', 'Nervous']

In [ ]:
# Find indices where all columns have zero values for data_1
data_1_zero_indices = data_1.index[data_1[emotions_columns].eq(0).all(axis=1)].tolist()

# Find indices where all columns have zero values for data_2
data_2_zero_indices = data_2.index[data_2[emotions_columns].eq(0).all(axis=1)].tolist()


In [ ]:
# Find common zero indices
need_delete_indices = set(data_1_zero_indices + data_2_zero_indices)
len(need_delete_indices)

In [ ]:
data_1 = data_1.drop(need_delete_indices)
data_2 = data_2.drop(need_delete_indices)

In [ ]:
data_1

In [ ]:
#get all mean values
for column in emotions_columns:
    mean_value = (data_1[column] + data_2[column]) / 2
    data_1[column] = mean_value


In [ ]:
data=data_1

In [ ]:
#Select interesting columns
data.head()

In [ ]:
len(data.index)

In [ ]:
#check missing values
data.isnull().sum()

In [ ]:
data['OriginalText']=data['full_text'] #copy tweets for the embedding

In [ ]:
#Calculate positive_affect, negative_affect and SWB
column_names = ['Attentive', 'Alert', 'Determined', 'Inspired', 'Active']
data["Positive_affect"]= data[column_names].sum(axis=1)

column_names = ['Hostile', 'Ashamed', 'Upset', 'Afraid', 'Nervous']
data["Negative_affect"]= data[column_names].sum(axis=1)

data["SWB"]= data["Positive_affect"] - data["Negative_affect"]

In [ ]:
#Delete columns that won't be used.
data = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'in_reply_to_user_id', 'geo', 'coordinates',
                  'place', 'favorited', 'retweeted', 'quoted_status', 'date',
                  'quoted_status_id', 'entities','withheld_in_countries'], axis='columns')

data = data.reset_index(drop=True)


In [ ]:
#transforming dtypes
data['created_at'] = pd.to_datetime(data['created_at'], utc = True)
data['possibly_sensitive'] = data['possibly_sensitive'].astype('bool')
data['in_reply_to_status_id'] = data['in_reply_to_status_id'].astype(int)
data['is_reply'] = np.where(data['in_reply_to_status_id'] != 0, True, False)
data = data.drop(['in_reply_to_status_id'], axis='columns')
data['full_text'] = data['full_text'].astype(str)

#30/01/2020: WHO declared covid an international health emergency
#I assume this the start of the worldwide covid star
start_date = pd.to_datetime('2020-01-30', utc = True)
data['days_since_COVID19'] = (data['created_at'] - start_date).dt.days

#get hour in day it was tweeted
data['hour'] = data['created_at'].dt.hour

In [ ]:
data['url_count'] = data['full_text'].str.count("http") #url_count
data['mentions_count'] = data['full_text'].str.count("\@") #mentions_count

#REMOVE URLS
#remove usernames and links from text
def remove_usernames_links(tweet):
    tweet = re.sub(r'^RT', '', tweet) #remove RT
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    tweet = re.sub(r'\s+', ' ', tweet) #remove multiple spaces
    return tweet
data['full_text'] = data['full_text'].apply(remove_usernames_links)

In [ ]:
#remove trailing whitespace
data['full_text'] = data['full_text'].str.strip()

In [ ]:
#make some extra columns that could add predictive value to the model
data["questionmark_count"] = data['full_text'].str.count("\?") #questionmark_count
data["exclamationmark_count"] = data['full_text'].str.count("\!") #exclamationmark_count
data["points_count"] = data['full_text'].str.count("\.") #points_count
data['uppercase_ratio'] = (data['full_text'].str.findall(r'[A-Z]').str.len())/(data['full_text'].str.len()) #uppercase_ratio
data['hashtag_count'] = data['full_text'].str.count("\#")  #hashtag_count


In [ ]:
#remove usernames and links from text
def remove_hash(tweet):
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet) #remove multiple spaces
    return tweet
data['full_text'] = data['full_text'].apply(remove_hash)

In [ ]:
def text_cleaning(text):
    #dealing with emojis
    tweet = emoji.demojize(text, delimiters=("", ""))

    #remove multiple spaces
    tweet = re.sub(r'\s+', ' ', tweet)

    #expanding abbreviations (part1)
    #creating an empty list
    expanded_words = []
    for word in tweet.split():
        # using contractions.fix to expand the shortened words
        expanded_words.append(contractions.fix(word))

    tweet = ' '.join(expanded_words)

    #expanding abbreviations (part2)
    abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
    }
    word_list = tweet.split()
    r_string = []
    for word in word_list:
        r_string.append(abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word)

    tweet = ' '.join(r_string)

    #character normalization
    tweet = re.sub(r'([A-Za-z])\1{2,}', r'\1', tweet)

    #remove punctuation and numbers
    string.punctuation
    tweet  = "".join([char for char in tweet if char not in string.punctuation])
    tweet = re.sub('[0-9]+', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet) #remove multiple spaces

    return tweet

data['clean_text'] = data['full_text'].apply(lambda x: text_cleaning(x))

In [ ]:
#convert text to lowercase
data['clean_text'] = data['clean_text'].str.lower()

#remove trailing whitespace
data['clean_text'] = data['clean_text'].str.strip()

In [ ]:
import spacy

# Load the English language model from spaCy
nlp = spacy.load("en_core_web_sm")

# Create a copy of the spaCy stop words set
custom_stop_words = set(nlp.Defaults.stop_words)
custom_stop_words.update({'rt','amp'}) #specific in tweets
# Words to remove from the custom stop words set (avoiding deleting negative words mainly from vander)
words_to_remove = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

# Remove specified words from the custom stop words set
custom_stop_words -= set(words_to_remove)
# Define the preprocessing function to tokenization, lemmatization, and stopwords
def preprocessing(text):
    # Apply spaCy for token stop words and lemma
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    filtered_tokens = [lemma for lemma in lemmatized_tokens if lemma.lower() not in custom_stop_words]

    return ' '.join(filtered_tokens)


In [ ]:
data['clean_text'] = data['clean_text'].apply(lambda x: preprocessing(x))

In [ ]:
data['clean_text'] = data['clean_text'].astype(str)

In [ ]:
data.head(30)

In [ ]:
#write to csv
data.to_csv("/content/drive/MyDrive/codeAndData/data/SA.csv", index=False)